In [1]:
import os
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import Dense,Dropout,Activation, Conv2D, MaxPool2D, Flatten
import numpy as np
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras import losses
from sklearn import metrics
import random
import pickle

Using TensorFlow backend.


In [2]:
from os import walk
current_path = os.getcwd()
mypath = os.path.join(current_path, 'all-mias_preprocessed_128')
f = []
for (dirpath, dirnames, filenames) in walk(mypath):
    f.extend(filenames)
    break

In [3]:
print(f)

['mdb001.pgm', 'mdb002.pgm', 'mdb005.pgm', 'mdb010.pgm', 'mdb012.pgm', 'mdb013.pgm', 'mdb015.pgm', 'mdb017.pgm', 'mdb019.pgm', 'mdb021.pgm', 'mdb023.pgm', 'mdb025.pgm', 'mdb028.pgm', 'mdb030.pgm', 'mdb032.pgm', 'mdb058.pgm', 'mdb063.pgm', 'mdb069.pgm', 'mdb072.pgm', 'mdb075.pgm', 'mdb080.pgm', 'mdb081.pgm', 'mdb083.pgm', 'mdb090.pgm', 'mdb091.pgm', 'mdb092.pgm', 'mdb095.pgm', 'mdb097.pgm', 'mdb099.pgm', 'mdb102.pgm', 'mdb104.pgm', 'mdb105.pgm', 'mdb107.pgm', 'mdb110.pgm', 'mdb111.pgm', 'mdb115.pgm', 'mdb117.pgm', 'mdb120.pgm', 'mdb121.pgm', 'mdb124.pgm', 'mdb125.pgm', 'mdb126.pgm', 'mdb127.pgm', 'mdb130.pgm', 'mdb132.pgm', 'mdb134.pgm', 'mdb141.pgm', 'mdb142.pgm', 'mdb144.pgm', 'mdb145.pgm', 'mdb148.pgm', 'mdb150.pgm', 'mdb152.pgm', 'mdb155.pgm', 'mdb158.pgm', 'mdb160.pgm', 'mdb163.pgm', 'mdb165.pgm', 'mdb167.pgm', 'mdb170.pgm', 'mdb171.pgm', 'mdb175.pgm', 'mdb178.pgm', 'mdb179.pgm', 'mdb181.pgm', 'mdb184.pgm', 'mdb186.pgm', 'mdb188.pgm', 'mdb190.pgm', 'mdb191.pgm', 'mdb193.pgm', 'mdb1

In [4]:
info=pd.read_csv("all-mias_preprocessed_128_info.txt",sep=" ")
info=info.drop('Unnamed: 7',axis=1)

In [5]:
info

,REFNUM,BG,CLASS,SEVERITY,X,Y,RADIUS
0,mdb001,G,CIRC,B,535.0,425.0,197.0
1,mdb002,G,CIRC,B,522.0,280.0,69.0
2,mdb003,D,NORM,NaN,NaN,NaN,NaN
3,mdb004,D,NORM,NaN,NaN,NaN,NaN
4,mdb005,F,CIRC,B,500.0,168.0,26.0
...,...,...,...,...,...,...,...
313,mdb318,D,NORM,NaN,NaN,NaN,NaN
314,mdb319,D,NORM,NaN,NaN,NaN,NaN
315,mdb320,D,NORM,NaN,NaN,NaN,NaN
316,mdb321,D,NORM,NaN,NaN,NaN,NaN


In [6]:
info.dropna(subset = ["SEVERITY"], inplace=True)

info.reset_index(inplace = True)
info

,index,REFNUM,BG,CLASS,SEVERITY,X,Y,RADIUS
0,0,mdb001,G,CIRC,B,535.0,425.0,197.0
1,1,mdb002,G,CIRC,B,522.0,280.0,69.0
2,4,mdb005,F,CIRC,B,500.0,168.0,26.0
3,9,mdb010,F,CIRC,B,525.0,425.0,33.0
4,11,mdb012,F,CIRC,B,471.0,458.0,40.0
...,...,...,...,...,...,...,...,...
106,269,mdb274,F,MISC,M,127.0,505.0,123.0
107,285,mdb290,D,CIRC,B,337.0,353.0,45.0
108,307,mdb312,F,MISC,B,240.0,263.0,20.0
109,309,mdb314,F,MISC,B,518.0,191.0,39.0


In [7]:
ids = {}
for i in range(len(info)):
    ids[i] = info.REFNUM[i]

In [8]:
label = []
for i in range(len(info)):
    if info.SEVERITY[i] == 'B':
        label.append(1)
    else:
        label.append(0)

In [9]:
label = np.array(label)

In [10]:
label.shape

(111,)

In [11]:
img_name = []

for i in range(len(label)):
        img_name.append(mypath +"\\" +info.REFNUM[i]+ '.pgm')

In [12]:
img_name = np.array(img_name)

In [13]:
print(type(img_name[1]))

<class 'numpy.str_'>


In [14]:
print(f'image addres amount {img_name.shape}')

image addres amount (111,)


In [15]:
img_path = []
last_label = []
for i in range(len(img_name)):
    
    img = cv2.imread(img_name[i], 0)
    img = cv2.resize(img, (128,128))
    rows, cols= img.shape
    for angle in range(360):
            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)    #Rotate 0 degree
            img_rotated = cv2.warpAffine(img, M, (128, 128))
            img_path.append(img_rotated)
            if label[i] == 1:
                last_label.append(1)
            else:
                last_label.append(0)

In [16]:
last_label = np.array(last_label)
img_path = np.array(img_path)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(img_path, last_label, test_size = 0.2, random_state = 42)

In [18]:
len(x_train),len(x_test),len(y_train),len(y_test)

(31968, 7992, 31968, 7992)

In [19]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [20]:
x_train.shape, x_test.shape

((31968, 128, 128), (7992, 128, 128))

In [21]:
(a,b,c)=x_train.shape # (35136, 224, 224)
x_train = np.reshape(x_train, (a, b, c, 1)) # 1 for gray scale
(a, b, c)=x_test.shape
x_test = np.reshape(x_test, (a, b, c, 1))

In [25]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(128, 128, 1)))
    model.add(Conv2D(64, kernel_size=(3,3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=(3,3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
  
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

In [26]:
model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 124, 124, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 30, 30, 64)        0         
_________________________________________________________________
dense_3 (Dense)              (None, 30, 30, 64)       

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

hist = model.fit(x_train,y_train,validation_split=0.2,epochs=10,batch_size=64)

Train on 25574 samples, validate on 6394 samples
Epoch 1/10
25574/25574 [==============================] - 1017s 40ms/step - loss: 1.3357 - accuracy: 0.5396 - val_loss: 0.6781 - val_accuracy: 0.5357
Epoch 2/10
25574/25574 [==============================] - 909s 36ms/step - loss: 0.6776 - accuracy: 0.5559 - val_loss: 0.6724 - val_accuracy: 0.5405
Epoch 3/10
25574/25574 [==============================] - 892s 35ms/step - loss: 0.6686 - accuracy: 0.5626 - val_loss: 0.6682 - val_accuracy: 0.5536
Epoch 4/10
25574/25574 [==============================] - 828s 32ms/step - loss: 0.6609 - accuracy: 0.5703 - val_loss: 0.6587 - val_accuracy: 0.5613
Epoch 5/10
25574/25574 [==============================] - 804s 31ms/step - loss: 0.6513 - accuracy: 0.5711 - val_loss: 0.6552 - val_accuracy: 0.5261
Epoch 6/10
25574/25574 [==============================] - 6170s 241ms/step - loss: 0.6404 - accuracy: 0.5787 - val_loss: 0.6389 - val_accuracy: 0.5621
Epoch 7/10
25574/25574 [==============================

In [ ]:
loss_value , accuracy = model.evaluate(x_test, y_test)

print('Test_loss_value = ' +str(loss_value))
print('test_accuracy = ' + str(accuracy))

print(model.predict(x_test))

In [ ]:
with open('bcc_cnn_128_360.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
y_pred